In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce

### 匯入資料
- 資料時間 : 2011 ~ 2020
- 預計欄位
 - 香蕉 前一日價格
 - 香蕉 平均價格
 - 香蕉 五日平均價格
 - 香蕉 成交量
 - 芭蕉 前日價格
 - 芭樂 前日價格
 - 節日(暫不納入)
 

#### 香蕉

In [10]:
df_banana = pd.read_csv("./cleaned data/價格_全國香蕉交易資料(初步清洗).csv", usecols=[0,1,2,6,8,10])
df_banana.head(10)

,日期,市場,產品,平均價(元/公斤),交易量(公斤),Year
0,2011-01-01,104 台北二,A1 香蕉,31.4,9645,2011
1,2011-01-02,104 台北二,A1 香蕉,32.0,12230,2011
2,2011-01-04,104 台北二,A1 香蕉,32.4,11976,2011
3,2011-01-05,104 台北二,A1 香蕉,33.2,7520,2011
4,2011-01-06,104 台北二,A1 香蕉,34.7,8160,2011
5,2011-01-07,104 台北二,A1 香蕉,35.5,7152,2011
6,2011-01-08,104 台北二,A1 香蕉,36.2,5850,2011
7,2011-01-09,104 台北二,A1 香蕉,35.4,8235,2011
8,2011-01-11,104 台北二,A1 香蕉,35.4,7320,2011
9,2011-01-12,104 台北二,A1 香蕉,35.7,5730,2011


In [11]:
df_banana.groupby("市場").sum()["交易量(公斤)"].sort_values(ascending=False)

市場
400 台中市    98946562
109 台北一    64851863
241 三重區    58371868
104 台北二    41543316
830 鳳山區    27701450
420 豐原區    15670654
800 高雄市    11969933
220 板橋區    10649034
600 嘉義市     6568129
260 宜蘭市     4708040
930 台東市     2437560
338 桃農       153565
423 東勢區       39878
Name: 交易量(公斤), dtype: int64

In [12]:
## 台北一市場
df_banana_TP = df_banana[df_banana["市場"] == "109 台北一"]
df_banana_TP.columns = ['日期', '市場', '產品', '香蕉_平均價', '香蕉_交易量', 'Year']
df_banana_TP.head()

,日期,市場,產品,香蕉_平均價,香蕉_交易量,Year
3145,2011-01-01,109 台北一,A1 香蕉,26.9,15779,2011
3146,2011-01-02,109 台北一,A1 香蕉,28.9,17462,2011
3147,2011-01-04,109 台北一,A1 香蕉,30.7,19405,2011
3148,2011-01-05,109 台北一,A1 香蕉,31.0,22050,2011
3149,2011-01-06,109 台北一,A1 香蕉,30.2,15062,2011


In [13]:
# 新欄位
df_banana_TP["香蕉_5日平均價格"] = round(df_banana_TP["香蕉_平均價"].rolling(5).mean().fillna(method="bfill"), 1)
df_banana_TP["香蕉_前1日平均價格"] = df_banana_TP["香蕉_平均價"].shift(1).fillna(method="bfill")

<ipython-input-13-1db77f364733>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_banana_TP["香蕉_5日平均價格"] = round(df_banana_TP["香蕉_平均價"].rolling(5).mean().fillna(method="bfill"), 1)
<ipython-input-13-1db77f364733>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_banana_TP["香蕉_前1日平均價格"] = df_banana_TP["香蕉_平均價"].shift(1).fillna(method="bfill")


In [14]:
df_banana_TP["日期"] = pd.to_datetime(df_banana_TP["日期"])

<ipython-input-14-e04fc00c08a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_banana_TP["日期"] = pd.to_datetime(df_banana_TP["日期"])


In [15]:
# index設為日期
df_banana_TP = df_banana_TP.set_index("日期")
df_banana_TP.count()

市場            3145
產品            3145
香蕉_平均價        3145
香蕉_交易量        3145
Year          3145
香蕉_5日平均價格     3145
香蕉_前1日平均價格    3145
dtype: int64

In [17]:
# resample
df_banana_TP = df_banana_TP.resample("D").ffill()
df_banana_TP.head()


,市場,產品,香蕉_平均價,香蕉_交易量,Year,香蕉_5日平均價格,香蕉_前1日平均價格
日期,,,,,,,
2011-01-01,109 台北一,A1 香蕉,26.9,15779,2011,29.5,26.9
2011-01-02,109 台北一,A1 香蕉,28.9,17462,2011,29.5,26.9
2011-01-03,109 台北一,A1 香蕉,28.9,17462,2011,29.5,26.9
2011-01-04,109 台北一,A1 香蕉,30.7,19405,2011,29.5,28.9
2011-01-05,109 台北一,A1 香蕉,31.0,22050,2011,29.5,30.7


#### 芭蕉

In [18]:
df_SecretBanana = pd.read_csv("../data/farmproduct_芭蕉_crawler.csv", usecols=[1,2,3,7,9])

In [19]:
target_cols = ['市場', '產品','平均價(元/公斤)','交易量(公斤)']
df_SecretBanana[target_cols] = df_SecretBanana[target_cols].apply(lambda x: x.str.strip(" "))
df_SecretBanana = df_SecretBanana[~(df_SecretBanana["市場"] == "A2 香蕉 芭蕉紅芭蕉")]
df_SecretBanana.head()

,日期,市場,產品,平均價(元/公斤),交易量(公斤)
0,100/01/01,104 台北二,A2 香蕉 芭蕉紅芭蕉,36.5,"1,143"
1,100/01/02,104 台北二,A2 香蕉 芭蕉紅芭蕉,50.0,782
2,100/01/04,104 台北二,A2 香蕉 芭蕉紅芭蕉,42.5,"1,427"
3,100/01/05,104 台北二,A2 香蕉 芭蕉紅芭蕉,34.8,"1,588"
4,100/01/06,104 台北二,A2 香蕉 芭蕉紅芭蕉,34.1,932


In [20]:
df_SecretBanana["日期"] = df_SecretBanana["日期"].apply(lambda x: re.sub("\d{3}", "{}".format((int(x.split("/")[0]) + 1911)), x))
df_SecretBanana["平均價(元/公斤)"] = df_SecretBanana["平均價(元/公斤)"].astype("float")
df_SecretBanana["交易量(公斤)"] = df_SecretBanana["交易量(公斤)"].apply(lambda x:re.sub(",","",x)).astype("int")

In [21]:
### 台北一市場
# 筆數較多是因為爬蟲時間不一樣所致
df_SecretBanana_TP = df_SecretBanana[df_SecretBanana["市場"] == "109 台北一"]
df_SecretBanana_TP = df_SecretBanana_TP[["日期", "平均價(元/公斤)"]]


In [22]:
df_SecretBanana_TP["日期"] = pd.to_datetime(df_SecretBanana_TP["日期"])
df_SecretBanana_TP.set_index("日期",inplace=True)
df_SecretBanana_TP["平均價(元/公斤)"] = df_SecretBanana_TP["平均價(元/公斤)"].shift(1).fillna(method="bfill")
                                                                       

In [23]:
df_SecretBanana_TP.columns = ["芭蕉_前1日平均價"]

In [25]:
df_SecretBanana_TP = df_SecretBanana_TP.resample("D").ffill()
df_SecretBanana_TP.head()

,芭蕉_前1日平均價
日期,
2011-01-01,39.0
2011-01-02,39.0
2011-01-03,39.0
2011-01-04,47.0
2011-01-05,40.1


### 芭樂

In [26]:
df_grava = pd.read_csv("../data/farmproduct_grava_crawler.csv", usecols=[0,1,6])
df_grava.head()

,日期,市場,平均價(元/公斤)
0,100/01/01,104 台北二,45.0
1,100/01/01,400 台中市,33.7
2,100/01/01,600 嘉義市,32.0
3,100/01/01,109 台北一,43.4
4,100/01/01,800 高雄市,28.0


In [27]:
df_grava_TP = df_grava[df_grava["市場"] == "109 台北一"]
df_grava_TP["日期"] = df_grava_TP["日期"].apply(lambda x: re.sub("\d{3}", "{}".format((int(x.split("/")[0]) + 1911)), x))
df_grava_TP["平均價(元/公斤)"] = df_grava_TP["平均價(元/公斤)"].shift(1).fillna(method="bfill") 

df_grava_TP["日期"] = pd.to_datetime(df_grava_TP["日期"])
df_grava_TP.set_index("日期", inplace=True)

df_grava_TP.drop(columns="市場",inplace=True)
df_grava_TP.columns = ["芭樂_前1日平均價格"]
df_grava_TP.head()

<ipython-input-27-ff5612cc8f06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grava_TP["日期"] = df_grava_TP["日期"].apply(lambda x: re.sub("\d{3}", "{}".format((int(x.split("/")[0]) + 1911)), x))
<ipython-input-27-ff5612cc8f06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grava_TP["平均價(元/公斤)"] = df_grava_TP["平均價(元/公斤)"].shift(1).fillna(method="bfill")
<ipython-input-27-ff5612cc8f06>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

,芭樂_前1日平均價格
日期,
2011-01-01,43.4
2011-01-02,43.4
2011-01-04,48.2
2011-01-05,43.6
2011-01-06,44.4


In [30]:
df_grava_TP = df_grava_TP.resample("D").ffill()
df_grava_TP.head()

,芭樂_前1日平均價格
日期,
2011-01-01,43.4
2011-01-02,43.4
2011-01-03,43.4
2011-01-04,48.2
2011-01-05,43.6


## 合併dataframes


In [31]:
df_banana_TP.head()

,市場,產品,香蕉_平均價,香蕉_交易量,Year,香蕉_5日平均價格,香蕉_前1日平均價格
日期,,,,,,,
2011-01-01,109 台北一,A1 香蕉,26.9,15779,2011,29.5,26.9
2011-01-02,109 台北一,A1 香蕉,28.9,17462,2011,29.5,26.9
2011-01-03,109 台北一,A1 香蕉,28.9,17462,2011,29.5,26.9
2011-01-04,109 台北一,A1 香蕉,30.7,19405,2011,29.5,28.9
2011-01-05,109 台北一,A1 香蕉,31.0,22050,2011,29.5,30.7


In [32]:
df_SecretBanana_TP.head()

,芭蕉_前1日平均價
日期,
2011-01-01,39.0
2011-01-02,39.0
2011-01-03,39.0
2011-01-04,47.0
2011-01-05,40.1


In [33]:
df_grava_TP.head()

,芭樂_前1日平均價格
日期,
2011-01-01,43.4
2011-01-02,43.4
2011-01-03,43.4
2011-01-04,48.2
2011-01-05,43.6


In [34]:
dfs = [df_banana_TP, df_SecretBanana_TP, df_grava_TP]
df_merged = reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), dfs)
df_merged.head()

,市場,產品,香蕉_平均價,香蕉_交易量,Year,香蕉_5日平均價格,香蕉_前1日平均價格,芭蕉_前1日平均價,芭樂_前1日平均價格
日期,,,,,,,,,
2011-01-01,109 台北一,A1 香蕉,26.9,15779,2011,29.5,26.9,39.0,43.4
2011-01-02,109 台北一,A1 香蕉,28.9,17462,2011,29.5,26.9,39.0,43.4
2011-01-03,109 台北一,A1 香蕉,28.9,17462,2011,29.5,26.9,39.0,43.4
2011-01-04,109 台北一,A1 香蕉,30.7,19405,2011,29.5,28.9,47.0,48.2
2011-01-05,109 台北一,A1 香蕉,31.0,22050,2011,29.5,30.7,40.1,43.6


In [35]:
df_merged.drop(columns="Year", inplace=True)

In [36]:
df_merged.to_csv("分析資料(0722).csv")

In [23]:
# 先讀取爬蟲下來的資料
# 再透過函式處理

def crawler_cleaner(df):
    cols = ['日期', '市場', '產品', '上價', '中價', '下價', '平均價(元/公斤)', '交易量(公斤)']
    df = df[cols]

    ## 時間
    # 改成西元年
    df["日期"] = df["日期"].apply(lambda x: re.sub("\d{3}", "{}".format((int(x.split("/")[0]) + 1911)), x))
    df["日期"] = pd.to_datetime(df["日期"])
    df = df.set_index("日期")

    # resample成每日
    # df = df.resample("D").fillna("ffill")#.interpolate()
    
    ## 清洗+補插值
    # 去除成交量中的雜質
    target_cols = df.select_dtypes("object")
    df[target_cols] = df[target_cols].apply(lambda x: x.str.strip(" "))
    
    #try:
    df["交易量(公斤)"] = df["交易量(公斤)"].apply(lambda x:re.sub(",|-", "", x)).astype("int")
    #except:
     #   pass
    
    return df
    
